In [ ]:
import sys

from format_markdown import markdown

sys.path.insert(0, "..")

In [ ]:
from versions import (
    JUPYTER_LSP_VERSION,
    JUPYTERLAB_LSP_VERSION,
    JUPYTERLAB_NEXT_MAJOR_VERSION,
    JUPYTERLAB_VERSION,
    REQUIRED_JUPYTER_SERVER,
    REQUIRED_JUPYTERLAB,
    REQUIRED_PYTHON,
)

## Installation

In [ ]:
%%markdown
### Please Read This First

Delivering LSP features to your JupyterLab **requires** three pieces:

#### `jupyter-lsp`

- runs in your `jupyter_server` web application on your server to handle requests from
  the browser to _language servers_
- to run, you need:
  - `python {REQUIRED_PYTHON}`
  - `jupyter_server {REQUIRED_JUPYTER_SERVER}`

#### `jupyterlab-lsp`

- runs in your browser, as an extension to JupyterLab
- to install it, you need:
  - `jupyterlab {REQUIRED_JUPYTERLAB}`

#### Language Servers

- run on your server
  - probably in another language runtime than python
- some can be automatically [detected](./Language%20Servers.ipynb) if installed
- others also need to be [configured](./Configuring.ipynb#language_servers)

### Fast Paths

Here are two approaches based on Jupyter documentation. If these do not meet
your needs, try [The Harder Way](#The-Harder-Way).

In [ ]:
%%markdown
#### conda (minimal python)

```bash
conda create -c conda-forge -n lsp 'python {REQUIRED_PYTHON}' 'jupyterlab={JUPYTERLAB_VERSION}' 'jupyterlab-lsp={JUPYTERLAB_LSP_VERSION}' 'jupyter-lsp-python={JUPYTER_LSP_VERSION}'
# jupyter-lsp-python includes both the server extension (jupyter-lsp) and pyls third-party server (python-language-server)
# if you swap it with another pre-made bundle, jupyter-lsp-r, you will get the server extension and r-languageserver;
# alternatively, manually install a language server of your choice (see the table below).
conda activate lsp
```

Then run

```bash
jupyter lab
```

Your browser should open to your local server.

#### docker (data science)

This approach is based roughly on the
[Jupyter docker-stacks documentation](https://github.com/jupyter/docker-stacks/tree/master/examples/docker-compose/notebook),
which should be consulted for more about connecting volumes, passwords, and
other advanced features:

> Note: docker instructions were **not** updated for JupyterLab 3.0 and
> extension 3.0. Please consider submitting a PR to fix it.

In [ ]:
%%markdown
##### `Dockerfile`

```dockerfile
# This already contains the python, r, julia, latex, and nodejs runtimes
FROM jupyter/datascience-notebook@sha256:73a577b006b496e1a1c02f5be432f4aab969c456881c4789e0df77c89a0a60c2

RUN conda install --quiet --yes --freeze-installed -c conda-forge \
    'python-language-server' \
    'jupyterlab={JUPYTERLAB_VERSION}' \
    'r-languageserver' \
    'texlab' \
    'chktex' \
    'jupyter-lsp={JUPYTER_LSP_VERSION}' \
  && jupyter labextension install --no-build \
    '@krassowski/jupyterlab-lsp@{JUPYTERLAB_LSP_VERSION}' \
  && jupyter lab build --dev-build=False --minimize=True \
  && conda clean --all -f -y \
  && rm -rf \
    $CONDA_DIR/share/jupyter/lab/staging \
    /home/$NB_USER/.cache/yarn \
  && fix-permissions $CONDA_DIR \
  && fix-permissions /home/$NB_USER
```

##### `docker-compose.yml`

```yaml
version: '2'

services:
  lsp-lab:
    build: .
    ports:
      - '18888:8888'
```

##### Build and Start

```bash
docker-compose up
```

You should now be able to access `http://localhost:18888/lab`, using the `token`
provided in the log.

### The Harder Way

#### Get A Working JupyterLab environment

Refer to the official
[JupyterLab Installation Documentation](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html)
for your installation approach.

| pip            | conda            | pipenv | poetry | `*` |
| -------------- | ---------------- | ------ | ------ | --- |
| [lab][lab-pip] | [lab][lab-conda] | `*`    | `*`    | `*` |

> `*` PRs welcome!

Verify your lab works:

```bash
jupyter lab --version
jupyter lab
```

[lab-conda]:
  https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html#conda
[lab-pip]:
  https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html#pip

In [ ]:
%%markdown
#### Install Jupyter[Lab] LSP

##### conda

```bash
conda install jupyterlab-lsp={JUPYTERLAB_LSP_VERSION}
```

##### pip

```bash
pip install jupyterlab-lsp=={JUPYTERLAB_LSP_VERSION}
```

### Next Step: Language Servers

Now that you have `jupyterlab-lsp`, `jupyter-lsp` and all of their dependencies,
you'll need some language servers. See:

- [Language Servers](./Language%20Servers.ipynb) that will be found
  automatically once installed
- [configuring](./Configuring.ipynb) `jupyter-lsp` for more control over which
  servers to load